# Fine-Tuning Stage III: Data Pre-Processing
## Game Logs and Game Summaries

In [174]:
import pandas as pd 
import numpy as np 
import re 
from collections import defaultdict

In [175]:
# Read in training data 
df = pd.read_csv("/Users/laurenmanis/Desktop/Fall 2024/DS 5690 | Gen AI Models/training.csv").drop(columns='Unnamed: 0')
players = pd.read_csv('/Users/laurenmanis/Desktop/Fall 2024/DS 5690 | Gen AI Models/nfl_players.csv').drop(columns='Unnamed: 0')

df.head()

,game_log,game_summary
0,7-H.Butker kicks 65 yards from KC 35 to end zo...,Headline: Chiefs hold off Ravens 27-20 when re...
1,GAME. \n39-J.Bates kicks 65 yards from DET 35 ...,Headline: Montgomery’s 1-yard touchdown run in...
2,GAME\n10-B.Mann kicks 65 yards from PHI 35 to ...,Headline: Barkley scores 3 TDs as Eagles beat ...
3,GAME\n16-W.Reichard kicks 65 yards from MIN 35...,Headline:\nDarnold throws 2 TD passes and Van ...
4,GAME\n2-T.Bass kicks 65 yards from BUF 35 to e...,Headline: Cook scores 3 TDs to help Bills rout...


In [176]:
# Enumerate/assign game ids for each unique game 
df['game_id'] = range(1, len(df) + 1)
df.head()

,game_log,game_summary,game_id
0,7-H.Butker kicks 65 yards from KC 35 to end zo...,Headline: Chiefs hold off Ravens 27-20 when re...,1
1,GAME. \n39-J.Bates kicks 65 yards from DET 35 ...,Headline: Montgomery’s 1-yard touchdown run in...,2
2,GAME\n10-B.Mann kicks 65 yards from PHI 35 to ...,Headline: Barkley scores 3 TDs as Eagles beat ...,3
3,GAME\n16-W.Reichard kicks 65 yards from MIN 35...,Headline:\nDarnold throws 2 TD passes and Van ...,4
4,GAME\n2-T.Bass kicks 65 yards from BUF 35 to e...,Headline: Cook scores 3 TDs to help Bills rout...,5


In [177]:
# Filter relevant players (QBs, RBs, Ks)
relevant_positions = ["QB", "RB", "K", "P"]
relevant_players = players[players["position"].isin(relevant_positions)].copy()
relevant_players['full_name'] = relevant_players['full_name'].replace("'", "")

# Remove apostrophes from names and create a `player_key` column
relevant_players["formatted_name"] = (
    relevant_players["jersey_number"].astype(str) + "-" +  
    relevant_players["full_name"].str.split(" ").str[0].str[0] + "." +  
    relevant_players["full_name"].str.split(" ").str[-1] 
)

# Create a mapping of player to team
player_team_map = relevant_players.set_index("formatted_name")["team"].to_dict()


In [178]:
# Function to separate out plays
def parse_plays(game_log):
    timestamp_pattern = r"\(\d{1,2}:\d{2}\)|\(:\d{1,2}\)"
    plays = []

    # Split the game log by timestamps, keeping the delimiters
    segments = re.split(f"({timestamp_pattern})", game_log)
    segments = [seg.strip() for seg in segments if seg.strip()]

    current_play = None

    for i in range(0, len(segments), 2):
        if i + 1 < len(segments):
            # Extract play text and timestamp
            play_text = segments[i]
            timestamp = segments[i + 1]

            # If there's a current play, append it to the list
            if current_play:
                plays.append(current_play)

            # Start a new play
            current_play = {"play": play_text, "timestamp": timestamp}
        else:
            # Append any remaining text to the current play
            if current_play:
                current_play["play"] += f" {segments[i]}"

    # Append the last play if it exists
    if current_play:
        plays.append(current_play)

    return plays

df['plays'] = df['game_log'].apply(parse_plays)

# Flatten the DataFrame into rows for each play
plays_df = df.explode('plays', ignore_index=True)

# Expand the 'plays' column into separate columns
plays_df = pd.concat([plays_df.drop(columns=['plays']), plays_df['plays'].apply(pd.Series)], axis=1)

plays_df.head()


,game_log,game_summary,game_id,play,timestamp
0,7-H.Butker kicks 65 yards from KC 35 to end zo...,Headline: Chiefs hold off Ravens 27-20 when re...,1,7-H.Butker kicks 65 yards from KC 35 to end zo...,(15:00)
1,7-H.Butker kicks 65 yards from KC 35 to end zo...,Headline: Chiefs hold off Ravens 27-20 when re...,1,(Shotgun) 22-D.Henry left end to BAL 32 for 2 ...,(14:19)
2,7-H.Butker kicks 65 yards from KC 35 to end zo...,Headline: Chiefs hold off Ravens 27-20 when re...,1,(Shotgun) 8-L.Jackson pass short right to 4-Z....,(13:55)
3,7-H.Butker kicks 65 yards from KC 35 to end zo...,Headline: Chiefs hold off Ravens 27-20 when re...,1,(Shotgun) 8-L.Jackson pass short right to 43-J...,(13:20)
4,7-H.Butker kicks 65 yards from KC 35 to end zo...,Headline: Chiefs hold off Ravens 27-20 when re...,1,(Shotgun) 8-L.Jackson pass short right to 43-J...,(12:43)


In [179]:
# Define a function to extract the first player in the play
def extract_first_player(play_text):
    # Remove apostrophes to avoid confusion
    play_text = play_text.replace("'", "")

    # Match the player format (e.g., "8-A.Rodgers")
    player_match = re.search(r"(\d+-[A-Z]\.[A-Za-z]+)", play_text)
    if player_match:
        return player_match.group(1)  
    return None 

# Define a function to map the extracted player to their team
def parse_and_map_teams(play_text, player_team_map):
    first_player = extract_first_player(play_text)  
    if first_player:
        return player_team_map.get(first_player, "Unknown Team")  
    return "Unknown Team" 

# Apply the function to identify the offense team
plays_df["offense_team"] = plays_df["play"].apply(lambda x: parse_and_map_teams(x, player_team_map))
plays_df.sample(5)


,game_log,game_summary,game_id,play,timestamp,offense_team
6714,GAME\n13-B.Pinion kicks 65 yards from ATL 35 t...,Headline: Cousins throws for 276 yards and 4 T...,46,(Shotgun) 6-B.Mayfield pass deep right to 10-T...,(1:47),TB
5680,GAME\n39-J.Bates kicks 63 yards from DET 35 to...,Headline: Jared Goff and Lions score at will i...,39,10-M.Jones pass short right to 1-T.Etienne to ...,(6:01),JAX
4963,GAME\n2-E.McPherson kicks 65 yards from CIN 35...,Headline: Burrow throws 5 TD passes as Bengals...,34,(Shotgun) 9-J.Burrow pass short right intended...,(13:12),CIN
1381,GAME\n7-H.Butker kicks 65 yards from KC 35 to ...,Headline: Mahomes throws 2 touchdown passes an...,10,(Shotgun) 15-P.Mahomes pass short left to 4-R....,(8:45),KC
4521,GAME\n9-C.Boswell kicks 65 yards from PIT 35 t...,Headline: Justin Fields leads Steelers past Br...,31,10-B.Nix pass short right to 33-J.Williams to ...,(:39),DEN


In [180]:
# Function to extract all the play-level stats
def extract_team_stats(play_text):
    stats = {
        "points": 0,
        "touchdowns": 0,
        "field_goals": 0,
        "missed_field_goals": 0,
        "rushing_yards": 0,
        "passing_yards": 0,
        "sacks": 0,
        "completions": 0,
        "incompletions": 0,
        "receptions": 0,
        "carries": 0,
        "plays": 1,
        "QB": None,
        "ball_carrier": None,
        "turnovers": 0,
        "nullified": 0,
        "new_drive": 0,
        "injury": None,
        "kicker": None
    }

    # Check for nullified plays
    if "- No Play" in play_text or "REVERSED" in play_text:
        stats["nullified"] = 1
        return stats

    # Touchdown detection
    if "TOUCHDOWN" in play_text:
        stats["touchdowns"] += 1
        if "extra point is GOOD" in play_text:
            stats["points"] += 7
        elif "TWO-POINT CONVERSION ATTEMPT" in play_text and "ATTEMPT FAILS" not in play_text:
            stats["points"] += 8
        else:
            stats["points"] += 6

    # Turnover detection
    if "INTERCEPTED" in play_text or "RECOVERED by" in play_text or "turnover" in play_text.lower():
        stats["turnovers"] += 1

    # Missed field goal detection
    if "field goal is No Good" in play_text:
        stats["missed_field_goals"] += 1
        kicker_match = re.search(r"(\d+-[\w.-]+)", play_text)
        stats["kicker"] = kicker_match.group(1) if kicker_match else None

    # Field goal success detection
    if "field goal is GOOD" in play_text:
        stats["field_goals"] += 1
        stats["points"] += 3
        kicker_match = re.search(r"(\d+-[\w.-]+)", play_text)
        stats["kicker"] = kicker_match.group(1) if kicker_match else None

    # Passing stats
    pass_match = re.search(r"pass.*?to\s([\w.-]+).*?for (-?\d+) yards", play_text)
    if pass_match:
        stats["passing_yards"] += int(pass_match.group(2))
        stats["completions"] += 1
        qb_match = re.search(r"(\d+-[\w.-]+) pass", play_text)
        stats["QB"] = qb_match.group(1) if qb_match else None
        stats["ball_carrier"] = pass_match.group(1)

    if "pass incomplete" in play_text:
        stats["incompletions"] += 1
        qb_match = re.search(r"(\d+-[\w.-]+) pass", play_text)
        stats["QB"] = qb_match.group(1) if qb_match else None

    # Rushing stats
    rush_match = re.search(r"(\d+-[\w.-]+).*?(left|right|middle).*?for (-?\d+) yards", play_text)
    if rush_match and "pass" not in play_text:
        stats["rushing_yards"] += int(rush_match.group(3))
        stats["carries"] += 1
        stats["ball_carrier"] = rush_match.group(1)

    # Sacks
    sack_match = re.search(r"sacked.*?for (-\d+) yards", play_text)
    if sack_match:
        stats["sacks"] += 1
        stats["passing_yards"] += int(sack_match.group(1))
        qb_match = re.search(r"(\d+-[\w.-]+) sacked", play_text)
        stats["QB"] = qb_match.group(1) if qb_match else None

    # Injury detection
    injury_match = re.search(r"([\w.-]+) was injured during the play", play_text)
    if injury_match:
        injured_player = injury_match.group(1)
        stats["injury"] = injured_player

    return stats

In [181]:
# Apply the function to extract stats and players
plays_df['team_stats'] = plays_df['play'].apply(extract_team_stats)

# Expand the stats dictionary into separate columns
team_stats_df = pd.json_normalize(plays_df['team_stats'])

# Concatenate the expanded stats back to the original DataFrame
plays_df = pd.concat([plays_df.drop(columns=['team_stats']), team_stats_df], axis=1)
plays_df.head()


,game_log,game_summary,game_id,play,timestamp,offense_team,points,touchdowns,field_goals,missed_field_goals,...,receptions,carries,plays,QB,ball_carrier,turnovers,nullified,new_drive,injury,kicker
0,7-H.Butker kicks 65 yards from KC 35 to end zo...,Headline: Chiefs hold off Ravens 27-20 when re...,1,7-H.Butker kicks 65 yards from KC 35 to end zo...,(15:00),Unknown Team,0,0,0,0,...,0,0,1,None,None,0,0,0,None,None
1,7-H.Butker kicks 65 yards from KC 35 to end zo...,Headline: Chiefs hold off Ravens 27-20 when re...,1,(Shotgun) 22-D.Henry left end to BAL 32 for 2 ...,(14:19),BAL,0,0,0,0,...,0,1,1,None,22-D.Henry,0,0,0,None,None
2,7-H.Butker kicks 65 yards from KC 35 to end zo...,Headline: Chiefs hold off Ravens 27-20 when re...,1,(Shotgun) 8-L.Jackson pass short right to 4-Z....,(13:55),BAL,0,0,0,0,...,0,0,1,None,None,0,1,0,None,None
3,7-H.Butker kicks 65 yards from KC 35 to end zo...,Headline: Chiefs hold off Ravens 27-20 when re...,1,(Shotgun) 8-L.Jackson pass short right to 43-J...,(13:20),BAL,0,0,0,0,...,0,0,1,8-L.Jackson,43-J.Hill,0,0,0,None,None
4,7-H.Butker kicks 65 yards from KC 35 to end zo...,Headline: Chiefs hold off Ravens 27-20 when re...,1,(Shotgun) 8-L.Jackson pass short right to 43-J...,(12:43),BAL,0,0,0,0,...,0,0,1,8-L.Jackson,43-J.Hill,0,0,0,None,None


In [182]:
# Function to group plays into drives
def assign_drive_numbers(plays_df):
    plays_df["drive_number"] = 0
    plays_df["quarter_end"] = False  
    # Track previous team to identify turnovers on downs
    plays_df["previous_team"] = None  

    # Group by game_id and assign drive numbers within each game
    for game_id, group in plays_df.groupby("game_id"):
        drive_number = 0
        drive_numbers = []
        quarter_ends = []
        previous_team = None
        consecutive_team_plays = 0

        for _, play in group.iterrows():
            current_team = play["offense_team"]

            # Assign the current drive number
            drive_numbers.append(drive_number)

            # Mark if the current play is an "END QUARTER"
            is_quarter_end = "END QUARTER" in play["play"]
            quarter_ends.append(is_quarter_end)

            # Check if the play continues the current drive
            if current_team == previous_team:
                consecutive_team_plays += 1
            else:
                consecutive_team_plays = 1 

            # Check for turnover on downs: 4 consecutive plays with no points scored, followed by a change in team
            if consecutive_team_plays >= 4 and current_team != previous_team:
                drive_number += 1
                consecutive_team_plays = 1 

            # Check for other drive-ending conditions
            if (
                "extra point" in play["play"].lower()
                or "field goal" in play["play"].lower()
                or "two-point conversion attempt" in play["play"].lower()
                or "punt" in play["play"].lower()
                or "turnover" in play["play"].lower()
                or "recovered by" in play["play"].lower()
                or "INTERCEPTED" in play["play"]
                or "Touchback" in play["play"]
                or "END QUARTER 2" in play["play"]
            ):
                drive_number += 1

            # Update the previous team
            previous_team = current_team

        # Assign drive numbers and quarter end markers back to the DataFrame
        plays_df.loc[group.index, "drive_number"] = drive_numbers
        plays_df.loc[group.index, "quarter_end"] = quarter_ends

    # Drop the temporary column used for tracking the previous team
    plays_df.drop(columns=["previous_team"], inplace=True)

    return plays_df

# Apply the function
plays_df = assign_drive_numbers(plays_df)
plays_df.head()


,game_log,game_summary,game_id,play,timestamp,offense_team,points,touchdowns,field_goals,missed_field_goals,...,plays,QB,ball_carrier,turnovers,nullified,new_drive,injury,kicker,drive_number,quarter_end
0,7-H.Butker kicks 65 yards from KC 35 to end zo...,Headline: Chiefs hold off Ravens 27-20 when re...,1,7-H.Butker kicks 65 yards from KC 35 to end zo...,(15:00),Unknown Team,0,0,0,0,...,1,None,None,0,0,0,None,None,0,False
1,7-H.Butker kicks 65 yards from KC 35 to end zo...,Headline: Chiefs hold off Ravens 27-20 when re...,1,(Shotgun) 22-D.Henry left end to BAL 32 for 2 ...,(14:19),BAL,0,0,0,0,...,1,None,22-D.Henry,0,0,0,None,None,1,False
2,7-H.Butker kicks 65 yards from KC 35 to end zo...,Headline: Chiefs hold off Ravens 27-20 when re...,1,(Shotgun) 8-L.Jackson pass short right to 4-Z....,(13:55),BAL,0,0,0,0,...,1,None,None,0,1,0,None,None,1,False
3,7-H.Butker kicks 65 yards from KC 35 to end zo...,Headline: Chiefs hold off Ravens 27-20 when re...,1,(Shotgun) 8-L.Jackson pass short right to 43-J...,(13:20),BAL,0,0,0,0,...,1,8-L.Jackson,43-J.Hill,0,0,0,None,None,1,False
4,7-H.Butker kicks 65 yards from KC 35 to end zo...,Headline: Chiefs hold off Ravens 27-20 when re...,1,(Shotgun) 8-L.Jackson pass short right to 43-J...,(12:43),BAL,0,0,0,0,...,1,8-L.Jackson,43-J.Hill,0,0,0,None,None,1,False


In [183]:
# Fill "Unknown Team" with the nearest non-Unknown team within the same game and drive 
def fill_unknown_offense_team(df):
    # Sort the DataFrame to ensure logical order
    df = df.sort_values(by=['game_id', 'drive_number']).reset_index(drop=True)

    # Replace 'Unknown Team' with NaN for filling
    df['offense_team'] = df['offense_team'].replace('Unknown Team', None)

    # Forward fill and backward fill within each game_id and drive_number
    df['offense_team'] = (
        df.groupby(['game_id', 'drive_number'])['offense_team']
        .apply(lambda group: group.ffill().bfill())
        .reset_index(level=[0, 1], drop=True)  
    )

    # Fill any remaining NaN values with 'Unknown Team' (Should just be kickoffs)
    df['offense_team'] = df['offense_team'].fillna('Unknown Team')

    return df

# Apply the function
plays_df = fill_unknown_offense_team(plays_df)
plays_df.head()


,game_log,game_summary,game_id,play,timestamp,offense_team,points,touchdowns,field_goals,missed_field_goals,...,plays,QB,ball_carrier,turnovers,nullified,new_drive,injury,kicker,drive_number,quarter_end
0,7-H.Butker kicks 65 yards from KC 35 to end zo...,Headline: Chiefs hold off Ravens 27-20 when re...,1,7-H.Butker kicks 65 yards from KC 35 to end zo...,(15:00),Unknown Team,0,0,0,0,...,1,None,None,0,0,0,None,None,0,False
1,7-H.Butker kicks 65 yards from KC 35 to end zo...,Headline: Chiefs hold off Ravens 27-20 when re...,1,(Shotgun) 22-D.Henry left end to BAL 32 for 2 ...,(14:19),BAL,0,0,0,0,...,1,None,22-D.Henry,0,0,0,None,None,1,False
2,7-H.Butker kicks 65 yards from KC 35 to end zo...,Headline: Chiefs hold off Ravens 27-20 when re...,1,(Shotgun) 8-L.Jackson pass short right to 4-Z....,(13:55),BAL,0,0,0,0,...,1,None,None,0,1,0,None,None,1,False
3,7-H.Butker kicks 65 yards from KC 35 to end zo...,Headline: Chiefs hold off Ravens 27-20 when re...,1,(Shotgun) 8-L.Jackson pass short right to 43-J...,(13:20),BAL,0,0,0,0,...,1,8-L.Jackson,43-J.Hill,0,0,0,None,None,1,False
4,7-H.Butker kicks 65 yards from KC 35 to end zo...,Headline: Chiefs hold off Ravens 27-20 when re...,1,(Shotgun) 8-L.Jackson pass short right to 43-J...,(12:43),BAL,0,0,0,0,...,1,8-L.Jackson,43-J.Hill,0,0,0,None,None,1,False


In [184]:
# Ensure all values in the `play` column are strings
plays_df['play'] = plays_df['play'].astype(str)

# Create the play log for each drive
plays = plays_df[['game_id', 'drive_number', 'offense_team', 'play']].copy()

plays['play_log'] = plays.groupby(['game_id', 'drive_number'])['play'].transform(
    lambda x: f"Offense Team: {plays.loc[x.index[0], 'offense_team']}. " + " ".join(x)
)

# Drop duplicates to keep only one row per drive
plays = plays.drop_duplicates(subset=['game_id', 'drive_number'])
plays.head()

,game_id,drive_number,offense_team,play,play_log
0,1,0,Unknown Team,7-H.Butker kicks 65 yards from KC 35 to end zo...,Offense Team: Unknown Team. 7-H.Butker kicks 6...
1,1,1,BAL,(Shotgun) 22-D.Henry left end to BAL 32 for 2 ...,Offense Team: BAL. (Shotgun) 22-D.Henry left e...
14,1,2,KC,(Shotgun) 15-P.Mahomes pass short left to 4-R....,Offense Team: KC. (Shotgun) 15-P.Mahomes pass ...
19,1,3,BAL,8-L.Jackson pass short left to 80-I.Likely to ...,Offense Team: BAL. 8-L.Jackson pass short left...
24,1,4,KC,(Shotgun) 15-P.Mahomes pass short left to 10-I...,Offense Team: KC. (Shotgun) 15-P.Mahomes pass ...


In [185]:
# Save as a csv
plays_df.to_csv("/Users/laurenmanis/Desktop/Fall 2024/DS 5690 | Gen AI Models/player_plays.csv", index=False)

In [186]:
# Function to consolidate and create one row per drive
def consolidate_drive_stats(df):
    # Group by `game_id` and `drive_number`
    grouped = df.groupby(['game_id', 'drive_number'])

    # Initialize a list to store the consolidated rows
    consolidated_rows = []

    for (game_id, drive_number), group in grouped:
        # Initialize drive-level stats
        drive_stats = {
            "game_id": game_id,
            "drive_number": drive_number,
            "offense_team": group.iloc[0]['offense_team'],
            "points": group['points'].sum(),
            "touchdowns": group['touchdowns'].sum(),
            "field_goals": group['field_goals'].sum(),
            "missed_field_goals": group['missed_field_goals'].sum(),
            "rushing_yards": group['rushing_yards'].sum(),
            "passing_yards": group['passing_yards'].sum(),
            "sacks": group['sacks'].sum(),
            "completions": group['completions'].sum(),
            "incompletions": group['incompletions'].sum(),
            "turnovers": group['turnovers'].sum(),
            "plays": group['plays'].sum(),
            "QB": group['QB'].mode()[0] if not group['QB'].isna().all() else None,
            "quarter_end": group['quarter_end'].any(),  
        }

        # Collect unique ball carriers and their stats
        ball_carrier_data = group[['ball_carrier', 'rushing_yards', 'carries', 'receptions', 'plays']]
        unique_ball_carriers = ball_carrier_data.drop_duplicates(subset=['ball_carrier'])

        for i, row in enumerate(unique_ball_carriers.itertuples(), start=1):
            if pd.notna(row.ball_carrier):
                drive_stats[f'ball_carrier_{i}'] = row.ball_carrier
                drive_stats[f'rushing_yards_{i}'] = row.rushing_yards
                drive_stats[f'carries_{i}'] = row.carries
                drive_stats[f'receptions_{i}'] = row.receptions

        # Collect unique kickers and their stats
        kicker_data = group[['kicker', 'field_goals', 'missed_field_goals']]
        unique_kickers = kicker_data.drop_duplicates(subset=['kicker'])

        for i, row in enumerate(unique_kickers.itertuples(), start=1):
            if pd.notna(row.kicker):
                drive_stats[f'kicker_{i}'] = row.kicker
                drive_stats[f'field_goals_{i}'] = row.field_goals
                drive_stats[f'missed_field_goals_{i}'] = row.missed_field_goals

        # Consolidate injuries
        injuries = group['injury'].dropna().unique()
        drive_stats['injuries'] = "; ".join(injuries) if len(injuries) > 0 else None

        # Append consolidated row
        consolidated_rows.append(drive_stats)

    # Create a new DataFrame from the consolidated rows
    return pd.DataFrame(consolidated_rows)

# Apply the function to consolidate drive stats
consolidated_game_drives = consolidate_drive_stats(plays_df)
consolidated_game_drives.head()


,game_id,drive_number,offense_team,points,touchdowns,field_goals,missed_field_goals,rushing_yards,passing_yards,sacks,...,carries_10,receptions_10,ball_carrier_11,rushing_yards_11,carries_11,receptions_11,ball_carrier_12,rushing_yards_12,carries_12,receptions_12
0,1,0,Unknown Team,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,BAL,7,1,0,0,35,39,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2,KC,7,1,0,0,25,27,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,3,BAL,0,0,0,0,0,18,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,4,KC,0,0,0,0,0,32,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [187]:
# Generate drive summaries
def generate_drive_summary(row):
    # Extract values from the row's columns
    team_name = row["offense_team"]
    points_scored = row["points"]
    total_plays = row["plays"]
    total_rushing_yards = row["rushing_yards"]
    total_passing_yards = row["passing_yards"]
    total_turnovers = row["turnovers"]
    total_field_goals = row["field_goals"]
    total_missed_field_goals = row["missed_field_goals"]
    total_yards = total_rushing_yards + total_passing_yards
    injury = row.get("injuries", None)
    quarter_end = row.get("quarter_end", False)

    # Initialize player stats text
    player_stats = []

    # Add QB stats
    if not pd.isna(row["QB"]):
        player_stats.append(f"{row['QB']}: {row['passing_yards']} passing yards on {row['completions']} completions")

    # Dynamically extract ball carrier stats, excluding duplicates
    i = 1
    ball_carriers_set = set()
    while f"ball_carrier_{i}" in row:
        ball_carrier = row[f"ball_carrier_{i}"]
        if pd.notna(ball_carrier) and ball_carrier not in ball_carriers_set:
            ball_carriers_set.add(ball_carrier)
            rushing_yards = row.get(f"rushing_yards_{i}", 0)
            carries = row.get(f"carries_{i}", 0)
            receptions = row.get(f"receptions_{i}", 0)
            receiving_yards = row.get(f"receiving_yards_{i}", 0)
            touchdowns = row.get(f"player_touchdowns_{i}", 0)

            player_summary = f"{ball_carrier}: {rushing_yards} rushing yards on {carries} carries"
            if receptions > 0:
                player_summary += f", {receptions} receptions for {receiving_yards} receiving yards"
            if touchdowns > 0:
                player_summary += f", {touchdowns} touchdown(s)"

            player_stats.append(player_summary)
        i += 1

    player_stats_text = "; ".join(player_stats)

    # Handle field goal text
    field_goal_text = ""
    if total_field_goals > 0:
        field_goal_text = f"{team_name} made {total_field_goals} field goal(s)"
    if total_missed_field_goals > 0:
        missed_text = f"{team_name} missed {total_missed_field_goals} field goal(s)"
        field_goal_text = f"{field_goal_text}, and {missed_text}" if field_goal_text else missed_text

    # Handle injuries
    injury_text = f"Injury Report: {injury}" if injury else ""

    # Handle quarter end
    quarter_end_text = " This drive marked the end of the quarter." if quarter_end else ""

    # Generate the summary text
    output = f"""
    Drive Summary: There were {total_plays} plays in the drive, for a total of {total_yards} net yards gained. {team_name} scored {points_scored} points.
    Team Stats: {team_name} had {total_rushing_yards} rushing yards and {total_passing_yards} passing yards. They committed {total_turnovers} turnovers.
    {field_goal_text}
    Individual Stats: {player_stats_text}
    {injury_text}
    {quarter_end_text}
    """
    return output.strip()

# Apply the updated function to generate summaries
consolidated_game_drives["drive_summary"] = consolidated_game_drives.apply(generate_drive_summary, axis=1)
consolidated_game_drives["drive_summary"].head()


0    Drive Summary: There were 1 plays in the drive...
1    Drive Summary: There were 13 plays in the driv...
2    Drive Summary: There were 5 plays in the drive...
3    Drive Summary: There were 5 plays in the drive...
4    Drive Summary: There were 6 plays in the drive...
Name: drive_summary, dtype: object

In [188]:
# Merge play logs with drive summaries
game_drives = consolidated_game_drives[['game_id', 'drive_number', 'offense_team', 'drive_summary']].merge(
    plays[['game_id', 'drive_number', 'play_log']],  
    on=['game_id', 'drive_number'], 
    how='left'
)

# Combine play log and drive summary into a single field
game_drives['combined_summary'] = (
    "Play Log: " + game_drives['play_log'].fillna("No play log available") + "\n\n" + 
    "Drive Summary: " + game_drives['drive_summary']
)

# Drop unnecessary columns
game_drives = game_drives[['game_id', 'drive_number', 'offense_team', 'combined_summary']]
game_drives.head()

,game_id,drive_number,offense_team,combined_summary
0,1,0,Unknown Team,Play Log: Offense Team: Unknown Team. 7-H.Butk...
1,1,1,BAL,Play Log: Offense Team: BAL. (Shotgun) 22-D.He...
2,1,2,KC,Play Log: Offense Team: KC. (Shotgun) 15-P.Mah...
3,1,3,BAL,Play Log: Offense Team: BAL. 8-L.Jackson pass ...
4,1,4,KC,Play Log: Offense Team: KC. (Shotgun) 15-P.Mah...


In [189]:
# Join back to game summaries 
game_drives = game_drives.merge(plays_df[['game_id','game_summary']], on='game_id', how='left')
game_drives = game_drives.drop_duplicates()
game_drives.head()

,game_id,drive_number,offense_team,combined_summary,game_summary
0,1,0,Unknown Team,Play Log: Offense Team: Unknown Team. 7-H.Butk...,Headline: Chiefs hold off Ravens 27-20 when re...
147,1,1,BAL,Play Log: Offense Team: BAL. (Shotgun) 22-D.He...,Headline: Chiefs hold off Ravens 27-20 when re...
294,1,2,KC,Play Log: Offense Team: KC. (Shotgun) 15-P.Mah...,Headline: Chiefs hold off Ravens 27-20 when re...
441,1,3,BAL,Play Log: Offense Team: BAL. 8-L.Jackson pass ...,Headline: Chiefs hold off Ravens 27-20 when re...
588,1,4,KC,Play Log: Offense Team: KC. (Shotgun) 15-P.Mah...,Headline: Chiefs hold off Ravens 27-20 when re...


In [190]:
print("Training data consists of ",game_drives['game_id'].nunique()," NFL games and ", 
      len(game_drives[['game_id','drive_number']].drop_duplicates()), " unique drives", "for",
        game_drives['offense_team'].nunique(), "unique teams.")

Training data consists of  50  NFL games and  1127  unique drives for 32 unique teams.


In [191]:
# Save as a csv
game_drives.to_csv("/Users/laurenmanis/Desktop/Fall 2024/DS 5690 | Gen AI Models/drive_data.csv", index=False)

## Compute Game-Level Totals

In [192]:
# Group by game_id and drive_number to compute drive-level totals
drive_level_totals = plays_df.groupby(['game_id', 'drive_number','offense_team']).agg({
    'points': 'sum',
    'touchdowns': 'sum',
    'field_goals': 'sum',
    'missed_field_goals': 'sum',
    'rushing_yards': 'sum',
    'passing_yards': 'sum',
    'sacks': 'sum',
    'completions': 'sum',
    'incompletions': 'sum',
    'carries': 'sum',
    'plays': 'sum',
    'turnovers': 'sum',
}).reset_index()

drive_level_totals.head()


,game_id,drive_number,offense_team,points,touchdowns,field_goals,missed_field_goals,rushing_yards,passing_yards,sacks,completions,incompletions,carries,plays,turnovers
0,1,0,Unknown Team,0,0,0,0,0,0,0,0,0,0,1,0
1,1,1,BAL,7,1,0,0,35,39,0,3,1,6,13,0
2,1,2,KC,7,1,0,0,25,27,0,2,0,2,5,0
3,1,3,BAL,0,0,0,0,0,18,0,2,0,0,5,0
4,1,4,KC,0,0,0,0,0,32,1,2,0,0,6,0


In [193]:
# Group by game_id to compute game-level totals
game_level_totals = drive_level_totals.groupby(['game_id','offense_team']).agg({
    'points': 'sum',
    'touchdowns': 'sum',
    'field_goals': 'sum',
    'missed_field_goals': 'sum',
    'rushing_yards': 'sum',
    'passing_yards': 'sum',
    'sacks': 'sum',
    'completions': 'sum',
    'incompletions': 'sum',
    'carries': 'sum',
    'plays': 'sum',
    'turnovers': 'sum',
}).reset_index()

# Inspect the game-level totals
game_level_totals = game_level_totals[game_level_totals['offense_team'] != 'Unknown Team'].rename(columns={'offense_team':'team_player'})
game_level_totals.head()


,game_id,team_player,points,touchdowns,field_goals,missed_field_goals,rushing_yards,passing_yards,sacks,completions,incompletions,carries,plays,turnovers
0,1,BAL,20,2,2,1,182,264,1,24,13,29,85,1
1,1,KC,27,3,2,0,71,279,1,18,7,16,61,1
3,2,DET,19,2,2,0,156,199,2,17,9,26,69,1
4,2,LA,20,2,2,0,81,303,2,31,14,20,85,2
5,3,GB,16,1,3,1,129,181,2,15,17,16,73,1


In [194]:
# Player-level stats grouped by game_id, drive_number, and player
player_level_stats = plays_df.groupby(['game_id', 'drive_number', 'ball_carrier']).agg({
    'rushing_yards': 'sum',
    'carries': 'sum',
    'completions': 'sum',
    'passing_yards': 'sum',
    'touchdowns':'sum'
}).reset_index().rename(columns={"passing_yards":"receiving_yards","completions":"receptions"})

player_level_stats.head()


,game_id,drive_number,ball_carrier,rushing_yards,carries,receptions,receiving_yards,touchdowns
0,1,1,22-D.Henry,16,4,0,0,1
1,1,1,4-Z.Flowers,0,0,1,19,0
2,1,1,43-J.Hill,0,0,2,20,0
3,1,1,8-L.Jackson,19,2,0,0,0
4,1,2,1-X.Worthy,21,1,0,0,1


In [195]:
# Aggregate player stats for the entire game
game_player_totals = player_level_stats.groupby(['game_id', 'ball_carrier']).agg({
    'rushing_yards': 'sum',
    'carries': 'sum',
    'receptions': 'sum',
    'receiving_yards': 'sum',
    'touchdowns':'sum'
}).reset_index().rename(columns={'ball_carrier':'team_player'})

game_player_totals.head()


,game_id,team_player,rushing_yards,carries,receptions,receiving_yards,touchdowns
0,1,1-X.Worthy,21,1,2,47,2
1,1,10-I.Pacheco,43,12,2,33,0
2,1,15-N.Agholor,0,0,1,6,0
3,1,15-P.Mahomes,4,1,1,2,0
4,1,22-D.Henry,43,10,0,0,1


In [196]:
# QB-level stats grouped by game_id and drive_number
qb_level_stats = plays_df.groupby(['game_id', 'drive_number', 'QB']).agg({
    'passing_yards': 'sum',
    'completions': 'sum',
    'incompletions': 'sum',
    'sacks': 'sum',
    'turnovers': 'sum',  
    'touchdowns':'sum'
}).reset_index()

# Inspect the QB-level stats
qb_level_stats.head()


,game_id,drive_number,QB,passing_yards,completions,incompletions,sacks,turnovers,touchdowns
0,1,1,8-L.Jackson,39,3,1,0,0,0
1,1,2,15-P.Mahomes,27,2,0,0,0,0
2,1,3,8-L.Jackson,18,2,0,0,0,0
3,1,4,15-P.Mahomes,32,2,0,1,0,0
4,1,5,8-L.Jackson,-7,0,0,1,1,0


In [197]:
# Aggregate QB stats for the entire game
game_qb_totals = qb_level_stats.groupby(['game_id', 'QB']).agg({
    'passing_yards': 'sum',
    'completions': 'sum',
    'incompletions': 'sum',
    'sacks': 'sum',
    'turnovers': 'sum',  
    'touchdowns':'sum'
}).reset_index().rename(columns={'QB':'team_player'})

game_qb_totals.head()


,game_id,team_player,passing_yards,completions,incompletions,sacks,turnovers,touchdowns
0,1,15-P.Mahomes,279,18,7,1,0,1
1,1,8-L.Jackson,264,24,13,1,1,1
2,2,16-J.Goff,199,17,9,2,0,1
3,2,9-M.Stafford,303,31,14,2,1,1
4,3,1-J.Hurts,299,21,11,2,2,2


In [198]:
# Kicker-level stats grouped by game_id and drive_number
kicker_level_stats = plays_df.groupby(['game_id', 'drive_number', 'kicker']).agg({
    'field_goals': 'sum',
    'missed_field_goals': 'sum',
    'points': 'sum',  
    'plays': 'count'  
}).reset_index().rename(columns={"plays":"attempts"})

# Inspect the kicker-level stats
kicker_level_stats.head()


,game_id,drive_number,kicker,field_goals,missed_field_goals,points,attempts
0,1,6,7-H.Butker,1,0,3,1
1,1,7,7-H.Butker,1,0,3,1
2,1,8,9-J.Tucker,0,1,0,1
3,1,10,9-J.Tucker,1,0,3,1
4,1,17,9-J.Tucker,1,0,3,1


In [199]:
# Aggregate kicker stats for the entire game
game_kicker_totals = kicker_level_stats.groupby(['game_id', 'kicker']).agg({
    'field_goals': 'sum',
    'missed_field_goals': 'sum',
    'points': 'sum',  
    'attempts': 'count'  
}).reset_index().rename(columns={'kicker':'team_player'})

game_kicker_totals.head()


,game_id,team_player,field_goals,missed_field_goals,points,attempts
0,1,7-H.Butker,2,0,6,2
1,1,9-J.Tucker,2,1,6,3
2,2,16-J.Karty,2,0,6,2
3,2,39-J.Bates,2,0,6,2
4,3,4-J.Elliott,2,0,6,2


In [200]:
# Merge data frames 
team_results = game_level_totals
player_results = pd.concat([game_player_totals, game_qb_totals, game_kicker_totals], axis=0, ignore_index=True)
player_results.head()

,game_id,team_player,rushing_yards,carries,receptions,receiving_yards,touchdowns,passing_yards,completions,incompletions,sacks,turnovers,field_goals,missed_field_goals,points,attempts
0,1,1-X.Worthy,21.0,1.0,2.0,47.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10-I.Pacheco,43.0,12.0,2.0,33.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,15-N.Agholor,0.0,0.0,1.0,6.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,15-P.Mahomes,4.0,1.0,1.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,22-D.Henry,43.0,10.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [206]:
def create_team_numeric_summary(df):

    # Group by game_id and create a summary
    summaries = []
    for game_id, group in team_results.groupby("game_id"):
        summary = f"Game {game_id} Team Summary:\n"
        for _, team in group.iterrows():
            team_summary = (
                f"- {team['team_player']}: {team['points']} points, {team['touchdowns']} touchdowns, "
                f"{team['field_goals']} field goals, {team['rushing_yards']} rushing yards, "
                f"{team['passing_yards']} passing yards, {team['plays']} total plays, "
                f"{team['turnovers']} turnovers."
            )
            summary += team_summary + "\n"
        summaries.append({"game_id": game_id, "team_summary": summary.strip()})

    return pd.DataFrame(summaries)


team_numeric_summaries = create_team_numeric_summary(team_results)
team_numeric_summaries[0:3]

,game_id,team_summary
0,1,"Game 1 Team Summary:\n- BAL: 20 points, 2 touc..."
1,2,"Game 2 Team Summary:\n- DET: 19 points, 2 touc..."
2,3,"Game 3 Team Summary:\n- GB: 16 points, 1 touch..."


In [202]:
def create_player_numeric_summary(df):

    # Group by game_id and create a summary
    summaries = []
    for game_id, group in player_results.groupby("game_id"):
        summary = f"Game {game_id} Player Summary:\n"
        for _, player in group.iterrows():
            player_summary = f"- {player['team_player']}: "
            if player["passing_yards"] > 0:
                player_summary += f"{player['passing_yards']} passing yards, {player['completions']} completions; "
            if player["rushing_yards"] > 0:
                player_summary += f"{player['rushing_yards']} rushing yards on {player['carries']} carries; "
            if player["receptions"] > 0:
                player_summary += f"{player['receptions']} receptions for {player['receiving_yards']} yards; "
            if player["points"] > 0:
                player_summary += f"{player['points']} points; "
            summary += player_summary.strip("; ") + ".\n"
        summaries.append({"game_id": game_id, "player_summary": summary.strip()})

    return pd.DataFrame(summaries)


player_numeric_summaries = create_player_numeric_summary(player_results)
player_numeric_summaries[0:3]


,game_id,player_summary
0,1,Game 1 Player Summary:\n- 1-X.Worthy: 21.0 rus...
1,2,Game 2 Player Summary:\n- 10-C.Kupp: 9.0 rushi...
2,3,Game 3 Player Summary:\n- 1-J.Hurts: 32.0 rush...


In [210]:
# Merge summaries back into the drive logs dataframe
merged_drives = game_drives.merge(team_numeric_summaries, on="game_id", how="left").merge(player_numeric_summaries, on="game_id", how="left")

# Create a combined summary column
merged_drives["numeric_summary"] = (
    "Team Stats:\n" + merged_drives["team_summary"].fillna("") + "\n\n" +
    "Player Stats:\n" + merged_drives["player_summary"].fillna("")
)

merged_drives.head()

,game_id,drive_number,offense_team,combined_summary,game_summary,team_summary,player_summary,numeric_summary
0,1,0,Unknown Team,Play Log: Offense Team: Unknown Team. 7-H.Butk...,Headline: Chiefs hold off Ravens 27-20 when re...,"Game 1 Team Summary:\n- BAL: 20 points, 2 touc...",Game 1 Player Summary:\n- 1-X.Worthy: 21.0 rus...,Team Stats:\nGame 1 Team Summary:\n- BAL: 20 p...
1,1,1,BAL,Play Log: Offense Team: BAL. (Shotgun) 22-D.He...,Headline: Chiefs hold off Ravens 27-20 when re...,"Game 1 Team Summary:\n- BAL: 20 points, 2 touc...",Game 1 Player Summary:\n- 1-X.Worthy: 21.0 rus...,Team Stats:\nGame 1 Team Summary:\n- BAL: 20 p...
2,1,2,KC,Play Log: Offense Team: KC. (Shotgun) 15-P.Mah...,Headline: Chiefs hold off Ravens 27-20 when re...,"Game 1 Team Summary:\n- BAL: 20 points, 2 touc...",Game 1 Player Summary:\n- 1-X.Worthy: 21.0 rus...,Team Stats:\nGame 1 Team Summary:\n- BAL: 20 p...
3,1,3,BAL,Play Log: Offense Team: BAL. 8-L.Jackson pass ...,Headline: Chiefs hold off Ravens 27-20 when re...,"Game 1 Team Summary:\n- BAL: 20 points, 2 touc...",Game 1 Player Summary:\n- 1-X.Worthy: 21.0 rus...,Team Stats:\nGame 1 Team Summary:\n- BAL: 20 p...
4,1,4,KC,Play Log: Offense Team: KC. (Shotgun) 15-P.Mah...,Headline: Chiefs hold off Ravens 27-20 when re...,"Game 1 Team Summary:\n- BAL: 20 points, 2 touc...",Game 1 Player Summary:\n- 1-X.Worthy: 21.0 rus...,Team Stats:\nGame 1 Team Summary:\n- BAL: 20 p...


In [212]:
# Save as a csv
merged_drives.to_csv("/Users/laurenmanis/Desktop/Fall 2024/DS 5690 | Gen AI Models/game_drive_summaries.csv", index=False)